### Nettoyage et normalisation des données

#### 1. Charger les données

In [29]:
import pandas as pd

data = pd.read_csv('../data/raw/data.csv')

print('Les données ont été chargées')

Les données ont été chargées


#### 2. Supprimer les doublons

In [30]:
inital_len = len(data)

data = data.drop_duplicates()

current_lignes = len(data)

print(f"{inital_len - current_lignes} lignes ont été supprimées. {current_lignes} restants")

39 lignes ont été supprimées. 14601 restants


##### 3. Gérer les valeurs manquantes

- Identifier les valeurs manquantes

In [31]:
data.isna().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5444
negativereason_confidence        4100
airline                             0
name                                0
retweet_count                       0
text                                0
tweet_coord                     13586
tweet_created                       0
tweet_location                   4722
user_timezone                    4812
dtype: int64

- Garder les colonnes pertinents

In [32]:
data = data[['airline_sentiment', 'airline_sentiment_confidence', 'airline', 'text']]

##### 4. Traitement et Nettoyage de données

In [33]:
data["text"] = data["text"].fillna("").str.strip()                  # Debut et fin 
data["text"] = data["text"].str.replace(r"\s+", " ", regex=True)    # Espaces doublons

##### 4.1. Data 1: data_with_mentions

- Sauvegarder les données

In [34]:
data.to_csv('../data/processed/data_with_mentions.csv', index=False)

##### 4.2. Data 2: data_clean

- Supprimer les mentions 

In [35]:
data_2 = data.copy()

data_2["text"] = data_2["text"].str.replace(r'@[^\s]+', '', regex=True)

- Supprimer les espaces supplémentaires

In [36]:
data_2["text"] = data_2["text"].fillna("").str.strip()
data_2["text"] = data_2["text"].str.replace(r"\s+", " ", regex=True)

- Sauvegarder les données propres

In [37]:
data_2.to_csv('../data/processed/data_clean.csv', index=False)

##### 4.3. Data 3: data_balanced_undersampling

- Equilibrer les classes (UnderSampling)

In [38]:
data_3 = data.copy()

positive_class = data_3[data_3['airline_sentiment'] == 'positive']
neutral_class = data_3[data_3['airline_sentiment'] == 'neutral']
negative_class = data_3[data_3['airline_sentiment'] == 'negative']

print('--- Avant:')
print('\tPositive:', len(positive_class))
print('\tNeutral: ', len(neutral_class))
print('\tNegative:', len(negative_class))

frac = len(neutral_class) / len(negative_class)
balanced_negative = negative_class.sample(frac=frac)

print('\n--- Après:')
print('\tPositive:', len(positive_class))
print('\tNeutral: ', len(neutral_class))
print('\tNegative:', len(balanced_negative))

data_3 = positive_class.merge(neutral_class, 'outer').merge(balanced_negative, 'outer')

print('\n--- Total:', len(data_3))


--- Avant:
	Positive: 2353
	Neutral:  3091
	Negative: 9157

--- Après:
	Positive: 2353
	Neutral:  3091
	Negative: 3091

--- Total: 8535


- Sauvegarder les données propres

In [39]:
data_3.to_csv('../data/processed/data_balanced_undersampling.csv', index=False)